In [6]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix

In [14]:
train_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255., validation_split=0.15)
val_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

In [17]:
train = train_generator.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Data/train', 
                                            target_size=(224, 224), batch_size=32, class_mode='binary',
                                            subset='training', seed=42)
test = train_generator.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Data/train', 
                                          target_size=(224, 224), batch_size=32, class_mode='binary',
                                          subset='validation', seed=42)
val = val_generator.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Data/val', 
                                        target_size=(224, 224), batch_size=32, class_mode='binary')

Found 208 images belonging to 2 classes.
Found 36 images belonging to 2 classes.
Found 153 images belonging to 2 classes.


In [19]:
x_val = np.concatenate([val.next()[0] for i in range(val.__len__())])
y_val = np.concatenate([val.next()[1] for i in range(val.__len__())])

In [23]:
model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu',
                           input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    tf.keras.layers.Conv2D(filters=16, 
                           kernel_size=(5, 5),
                           padding='same',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')]
)

In [25]:
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_1.summary()
model_1.fit(train, validation_data=test, epochs=15)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 6)       168       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 6)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 16)      2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                1605664   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3

Посчитаем параметры модели

In [32]:
y_pred_1 = model_1.predict(x_val).round()
print(f'Accuracy: {accuracy_score(y_val, y_pred_1)}')
print('Confusion matrix')
print(confusion_matrix(y_val, y_pred_1))

Accuracy: 0.43790849673202614
Confusion matrix
[[40 29]
 [57 27]]


Получили слабенький результат, 
попробуем улучшить с помощью батч-нормализации

In [30]:
model_2 = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=6,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=16, 
                           kernel_size=(5, 5),
                           padding='same',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')]
)

In [31]:
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_2.summary()
model_2.fit(train, validation_data=test, epochs=15);

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 6)       168       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 112, 112, 6)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 6)       24        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 16)      2416      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 56, 56, 16)       

Посчитаем параметры модели

In [33]:
y_pred_2 = model_2.predict(x_val).round()
print(f'Accuracy: {accuracy_score(y_val, y_pred_2)}')
print('Confusion matrix')
print(confusion_matrix(y_val, y_pred_2))

Accuracy: 0.5359477124183006
Confusion matrix
[[25 44]
 [27 57]]


Батч_нормализация чуть улучшила результаты модели

Обучим ResNet 50 со случайными весами

In [34]:
train_generator_RS = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.15)
val_generator_RS = tf.keras.preprocessing.image.ImageDataGenerator()

train_RS = train_generator_RS.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Data/train', 
                                               target_size=(224, 224), batch_size=32, class_mode='binary', subset='training', seed=42)
test_RS = train_generator_RS.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Data/train', 
                                              target_size=(224, 224), batch_size=32, class_mode='binary', subset='validation', seed=42)
val_RS = val_generator_RS.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Data/val', 
                                           target_size=(224, 224), batch_size=32, class_mode='binary')

x_val_RS = np.concatenate([val_RS.next()[0] for i in range(val_RS.__len__())])
y_val_RS = np.concatenate([val_RS.next()[1] for i in range(val_RS.__len__())])

Found 208 images belonging to 2 classes.
Found 36 images belonging to 2 classes.
Found 153 images belonging to 2 classes.


In [36]:
i = tf.keras.layers.Input((224, 224, 3), dtype=tf.uint8)
o = tf.cast(i, tf.float32)
o = tf.keras.applications.resnet50.preprocess_input(o)
o = tf.keras.applications.ResNet50(include_top=False, weights=None)(o)
o = tf.keras.layers.Flatten()(o)
o = tf.keras.layers.Dense(1, activation='sigmoid')(o)

model_3 = tf.keras.Model(inputs=[i], outputs=[o])
model_3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_3.summary()
model_3.fit(train_RS, validation_data=test_RS, epochs=15);

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.cast (TFOpLambda)         (None, 224, 224, 3)       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 100353

Посмотрим результаты работы модели

In [37]:
y_pred_3 = model_3.predict(x_val_RS).round()
print(f'Accuracy: {accuracy_score(y_val_RS, y_pred_3)}')
print('Confusion matrix')
print(confusion_matrix(y_val_RS, y_pred_3))

Accuracy: 0.5032679738562091
Confusion matrix
[[18 52]
 [24 59]]


Плохое качество предсказания у модели

Обучим ResNet 50 с весами imagenet с запретом обучения

In [39]:
resnet50 = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')
resnet50.trainable = False

i = tf.keras.layers.Input(shape=(224, 224, 3), dtype=tf.uint8)
o = tf.cast(i, tf.float32)
o = tf.keras.applications.resnet50.preprocess_input(o)
o = resnet50(o)
o = tf.keras.layers.Flatten()(o)
o = tf.keras.layers.Dense(1, activation='sigmoid')(o)

model_4 = tf.keras.Model(inputs=[i], outputs=[o])
model_4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_4.summary()
model_4.fit(train_RS, validation_data=test_RS, epochs=10);

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.cast_1 (TFOpLambda)       (None, 224, 224, 3)       0         
_________________________________________________________________
tf.__operators__.getitem_1 ( (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add_1 (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
flatten_3 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1003

In [40]:
y_pred_4 = model_4.predict(x_val_RS).round()
print(f'Accuracy: {accuracy_score(y_val_RS, y_pred_4)}')
print('Confusion matrix')
print(confusion_matrix(y_val_RS, y_pred_4))

Accuracy: 0.5098039215686274
Confusion matrix
[[34 36]
 [39 44]]


Наличие предустановленныех весов не помогло улучшить результаты. Походу мало данных для обучения

Обучим ResNet 50 с весами imagenet без запрета обучения

In [42]:
resnet50_1 = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')
resnet50_1.trainable = True
i = tf.keras.layers.Input(shape=(224, 224, 3), dtype=tf.uint8)
o = tf.cast(i, tf.float32)
o = tf.keras.applications.resnet50.preprocess_input(o)
o = resnet50_1(o)
o = tf.keras.layers.Flatten()(o)
o = tf.keras.layers.Dense(1, activation='sigmoid')(o)

model_5 = tf.keras.Model(inputs=[i], outputs=[o])
model_5.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_5.summary()
model_5.fit(train_RS, validation_data=test_RS, epochs=10);

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.cast_3 (TFOpLambda)       (None, 224, 224, 3)       0         
_________________________________________________________________
tf.__operators__.getitem_2 ( (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add_2 (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
flatten_4 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 1003

In [43]:
y_pred_5 = model_5.predict(x_val_RS).round()
print(f'Accuracy: {accuracy_score(y_val_RS, y_pred_5)}')
print('Confusion matrix')
print(confusion_matrix(y_val_RS, y_pred_5))

Accuracy: 0.5490196078431373
Confusion matrix
[[ 9 61]
 [ 8 75]]


Получили самое лучшее значение Accuracy для этой выборки